In [1]:
import warnings
import requests
import base64
import time
import math
import copy
import json
import os
import io
import gc

from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaForCausalLM, BitsAndBytesConfig, AutoProcessor, PaliGemmaForConditionalGeneration, GPTQConfig
from exllamav2 import ExLlamaV2, ExLlamaV2Config, ExLlamaV2Cache, ExLlamaV2Tokenizer
from exllamav2.generator import ExLlamaV2Sampler, ExLlamaV2DynamicGenerator, ExLlamaV2BaseGenerator
from flask import Flask, render_template, request, jsonify
from scipy.special import softmax
from typing import Union, List
from llama_cpp import Llama
from tqdm import tqdm
from PIL import Image
import bitsandbytes
import numpy as np
import flash_attn
import random
import torch

/home/tobi/miniconda3/envs/assba/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/tobi/miniconda3/envs/assba/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [22]:
def use_llm(task_description, task_input, forced_beginning=None, directly_get_reply=True):
    inputs = {}
    inputs['chat'] = [
        {"role": "system", "content": task_description},
        {"role": "User", "content": task_input},
    ]
    if forced_beginning != None and forced_beginning.strip() != "":
        inputs['chat'].append({"role": "AI", "content": forced_beginning})
    inputs['model'] = "gemma-2-9b"
    inputs['model_dtype'] = "bfloat16"
    inputs['max_new_tokens'] = "512"
    inputs['debugmode'] = True
    inputs['images'] = []
    inputs['agent_task_mode'] = False
    inputs['use_functions'] = False
    inputs['use_voiceinput'] = False
    inputs['use_voiceoutput'] = False
    inputs['allow_imagegen'] = False
    inputs['beam_config'] = {
        "use_beam_search": False,
        "max_num_beams": "1",
        "depth_beams": "1",
        "min_conf_for_sure": "0.95",
        "min_conf_for_consider": "0.02",
        "prob_sum_for_search": "0.98",
    }

    
    url = f"http://127.0.0.1:10000/infer"
    response = requests.post(url, json=inputs) 
    if response.json()['status'] == "error":
        print(response.json()['error-info'])
        return json.dumps(response.json())
    ai_reply = response.json()['returned_content'][0]
    info = response.json()['info']
    
    returned_content = [
        {'role': 'AI', 'content': "\n".join(ai_reply)}
    ]
    if directly_get_reply:
        return returned_content[-1]['content']
    else:
        return json.dumps({'status': 'success', 'returned_content': returned_content, 'info': info})

In [18]:
out = use_llm(
    task_description = "You will be given a text. return it translated into english. reply in json format.", 
    task_input = "Am 14. Mai 2024 ist es soweit: Die Empfehlungen “Offene KI für alle!” werden als feierlicher Abschluss des Forums Offene KI in der Bildung bei Wikimedia Deutschland präsentiert. Dabei wird die Bildungswissenschaftlerin und Pädagogin Nele Hirsch mit verschiedenen Bildungs- und Digitalpolitiker*innen zu den Empfehlungen ins Gespräch kommen.",
    forced_beginning = "{\n    \"task_solution\": "
)
print(out)

"On May 14, 2024, the day has come: The recommendations 'Open KI for everyone!' will be presented as a festive conclusion to the Open KI in Education forum at Wikimedia Deutschland. The education scientist and pedagogue Nele Hirsch will talk to various education and digital politicians about the recommendations."
}


In [20]:
print(use_llm(
    task_description = "You will be given a text. return all words in the text. reply in json format.", 
    task_input = out,
    forced_beginning = "{\n    \"task_solution\": "
))

["On", "May", "14", "2024", "the", "day", "has", "come", "the", "recommendations", "Open", "KI", "for", "everyone", "will", "be", "presented", "as", "a", "festive", "conclusion", "to", "the", "Open", "KI", "in", "Education", "forum", "at", "Wikimedia", "Deutschland", "The", "education", "scientist", "and", "pedagogue", "Nele", "Hirsch", "will


In [43]:
print(use_llm(
    task_description = "You will be given a string. decide if it is an english noun / pronoun (\"task_solution\": true) or not (\"task_solution\": false). reply in json format.", 
    task_input = "\"narr\"",
    forced_beginning = "{\n    \"task_solution\": "
))

false
}
